In [39]:
import pandas as pd
import numpy as np

from nltk import pos_tag, word_tokenize
import re

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer



In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.gaussian_process import GaussianProcessClassifier
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

import nltk
import nltk as nlp
import string
import re

In [41]:
train = pd.read_csv('no_stopwords_combined.csv')
test = pd.read_csv('testnostopwords.csv')

train_copy = train.copy(deep = True)
test_copy = test.copy(deep = True)

In [42]:
train_copy.head(3)

,Unnamed: 0,Unnamed: 0.1,title,text,subject,date,target
0,0,0,Ramaphosa's ANC election win lifts South Afric...,johannesburg reuters south african banking sto...,worldnews,"December 19, 2017",1
1,1,1,VIGILANTE PIRATES INTERCEDE Where Government F...,like soldiers oden vigilante group reported fe...,Government News,"Apr 1, 2016",0
2,2,2,"SICK! DEMOCRAT ORGANIZER, Mayor DeBlasio Emplo...",last week huma abedin husband anthony weiner w...,left-news,"May 28, 2017",0


In [43]:
train_copy.isnull().sum()

Unnamed: 0        0
Unnamed: 0.1      0
title             0
text            718
subject           0
date              0
target            0
dtype: int64

### Starter Code EDA Result

- It is better to use the NaN rows, as all of them correspond to fake news
- One of the local rows in the dataset with null text is however labelled "true news"
- Pipeline doesn;t work with NaN values, so we'll replace NaN by 'Empty' string

In [44]:
#train_copy.dropna(subset = ['text'], inplace = True)

In [45]:
train_copy.fillna('Empty', inplace = True)

## Logistic Reg

In [46]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LogisticRegression())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

accuracy: 52.49%


In [47]:
print(confusion_matrix(test_copy['target'],
                      pred))

[[131 247]
 [135 291]]


## Support Vector Classifier

In [48]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

accuracy: 50.62%


In [49]:
print(confusion_matrix(test_copy['target'],
                      pred))

[[155 223]
 [174 252]]


## Multinomial Naive Bayes Classifier

In [50]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', LinearSVC())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

accuracy: 50.62%


In [51]:
print(confusion_matrix(test_copy['target'],
                      pred))

[[155 223]
 [174 252]]


## Gradient Boost Classifier

In [52]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', GradientBoostingClassifier(loss = 'deviance',
                                                   learning_rate = 0.01,
                                                   n_estimators = 10,
                                                   max_depth = 5,
                                                   random_state=42))])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 48.51%
[[330  48]
 [366  60]]


## XGBoost Classifier

In [53]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', XGBClassifier(loss = 'deviance',
                                        learning_rate = 0.01,
                                        n_estimators = 10,
                                        max_depth = 5,
                                        random_state=2020))])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 48.51%
[[330  48]
 [366  60]]


## Random Forest Classifier

In [55]:
pipe = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('model', RandomForestClassifier())])

model = pipe.fit(train_copy['text'],train_copy['target'])
pred = model.predict(test_copy['text'])

print("accuracy: {}%".format(round(accuracy_score(test_copy['target'],
                                                  pred)*100,2)))

print(confusion_matrix(test_copy['target'],
                      pred))

accuracy: 53.61%
[[ 29 349]
 [ 24 402]]


# LSTM - Recurrent Neural Nets

In [56]:
X = train_copy['text']
Y = train_copy['target']

le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [84]:
max_words = 500
max_len = 75

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X)

sequences = tok.texts_to_sequences(X)
sequences_matrix = sequence.pad_sequences(sequences, maxlen = max_len)

def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()

In [72]:
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(),
             metrics = ['accuracy'])

In [73]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 75)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 75, 50)            25000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [82]:
model.fit(sequences_matrix, Y, batch_size = 256, epochs = 10,
         validation_split = 0.2)

Train on 35918 samples, validate on 8980 samples
Epoch 1/10
35918/35918 [==============================] - 11s 294us/step - loss: 0.0416 - accuracy: 0.9856 - val_loss: 0.1085 - val_accuracy: 0.9728
Epoch 2/10
35918/35918 [==============================] - 11s 302us/step - loss: 0.0384 - accuracy: 0.9867 - val_loss: 0.0751 - val_accuracy: 0.9785
Epoch 3/10
35918/35918 [==============================] - 10s 284us/step - loss: 0.0370 - accuracy: 0.9880 - val_loss: 0.0629 - val_accuracy: 0.9815
Epoch 4/10
35918/35918 [==============================] - 17s 461us/step - loss: 0.0330 - accuracy: 0.9892 - val_loss: 0.0727 - val_accuracy: 0.9806
Epoch 5/10
35918/35918 [==============================] - 15s 406us/step - loss: 0.0297 - accuracy: 0.9899 - val_loss: 0.0653 - val_accuracy: 0.9801
Epoch 6/10
35918/35918 [==============================] - 12s 331us/step - loss: 0.0286 - accuracy: 0.9908 - val_loss: 0.0768 - val_accuracy: 0.9722
Epoch 7/10
35918/35918 [==============================] -

In [ ]:
test_sequences = tok.texts_to_sequences(test_copy['text'])
test_sequences_matrix = sequence.pad_sequences(test_sequences, 
                                               maxlen = max_len)

accr = model.evaluate(test_sequences_matrix, test_copy['target'])
print('Accuracy :{:0.2f}'.format(accr[1]))

In [ ]:
train_copy['text'][:20].str.len()